In [17]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import pickle
import tools
import sys
import subprocess
from collections import defaultdict
import whisper
module_path = os.path.abspath(os.path.join('./light'))
if module_path not in sys.path:
    sys.path.append(module_path)
from light.ASD import ASD as lightASD
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [18]:
model = talkNet()
#model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))
model.load_state_dict(torch.load("./exps/exp1/model/model51_0004.model"))
#model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))

windowSize = 51


09-29 15:56:05 Model para number = 15.01


In [19]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(windowSize,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(0)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=8,num_workers=14) #Cambiar num_workers

25 100


In [20]:
correctFrames, totalFrames = 0, 0
correctSamples, totalSamples = 0, 0
totalPreds = []
totalScores = []
for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
    with torch.no_grad():    
        audioFeature, visualFeature, labels = audioFeature.cuda(), visualFeature.cuda(), labels.cuda()
        predScores,predLabels = model((audioFeature,visualFeature)) #(audioFeature,visualFeature) si talknet
        batchPreds = torch.reshape(predLabels, labels.shape)
        #Precision a nivel de video
        videoPreds = torch.mode(batchPreds,dim=1)[0]
        totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
        for ten in torch.split(predScores[:,1],windowSize):
            totalScores.append(torch.mean(ten).detach().cpu().numpy())
        labelMode = torch.mode(labels,dim=1)[0]
        correctSamples += (videoPreds == labelMode).sum().float()
        totalSamples += len(labels)
        # Precision a nivel de frame
        labels = labels.reshape((-1))
        correctFrames += (predLabels == labels).sum().float()
        totalFrames += len(labels) 
#7865
print("Frame acc:", correctFrames/totalFrames)
print("Sample acc:", correctSamples/totalSamples)
# center = 47
# iAudio = tools.padAudio(audio,1,center,51,len(res[0])).unsqueeze(0)
# iVideo = tools.padVideo(res[0],center,51).unsqueeze(0)
# print(iAudio[40],item[0][40])
# print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
# print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
# print(audio[0],item[0][0])


from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score

df = pd.read_csv("testSamples.csv")
df2 = pd.DataFrame({'pred':totalPreds})
df3 = pd.DataFrame({'posScore':totalScores})
df["pred"] = df2
df["posScore"] = df3
df.to_csv("testPredsX.csv")
g_truth = pd.read_csv("testSamples.csv")["label"].values.tolist()
fpr, tpr, thresholds = metrics.roc_curve(g_truth, totalScores)
auc = metrics.roc_auc_score(g_truth, totalScores)
np.savetxt('cli.txt', df[df["label"]==1]["posScore"].values, fmt='%f')
np.savetxt('imp.txt', df[df["label"]==0]["posScore"].values, fmt='%f')
nCorrect = 0
for i in df[df["label"]==1]["posScore"].values:
    if i >=0.5:
        nCorrect +=1
print("AUC:", auc)

100%|██████████| 3751/3751 [03:40<00:00, 17.01it/s]


Frame acc: tensor(0.9324, device='cuda:0')
Sample acc: tensor(0.9323, device='cuda:0')
AUC: 0.9937767005133894
